<a href="https://colab.research.google.com/github/chetools/CHE4061_Fall2025/blob/main/VLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.7 MB/s eta 0:00:00


In [2]:
from importnb import Notebook
with Notebook():
    from che5 import Props

import numpy as np
import scipy as sp
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "plotly_dark"
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)

In [15]:
p=Props(['Benzene','Toluene'])
R=8.314 #J/(mol K)

In [4]:
z=np.array([0.4,0.6])

In [5]:
def bubbleP(x, T):
    Pi = x*p.Pvap(T)
    P=np.sum(Pi)
    return P, Pi/P

In [6]:
def dewP(y, T):
    Pisat = p.Pvap(T)
    P=1/np.sum(y/Pisat)
    return P, y*P/Pisat

In [7]:
xs= np.linspace(0,1,21)
T=350.
Ps = []
y1s = []
for x in np.c_[xs , 1-xs ]:
    P, y = bubbleP(x, T)
    Ps.append(P)
    y1s.append(y[0])
Ps = np.r_[Ps]
y1s=np.r_[y1s]

In [8]:
fig = make_subplots()
fig.add_scatter(x=xs,y=Ps, name='bubble', mode='lines')
fig.add_scatter(x=y1s,y=Ps, name='dew', mode='lines')
fig.update_layout(width=800,height=600, title=f'Benzene-Tolune VLE at T={T} K', xaxis_title='x, y', yaxis_title='Pressure (Pa)')

In [13]:
p.HvapNB

array([30761.43, 33179.83])

In [22]:
p.Tbn

array([353.24, 383.78])

In [27]:
def T_estimate(P, x):
    return np.sum(x*1/(1/p.Tbn-R*np.log(P/101325)/p.HvapNB))

In [26]:
T_estimate(1e5, z)

np.float64(371.0955479288866)

In [30]:
def bubbleT(x, P):
    T = sp.optimize.root_scalar(lambda T: bubbleP(x, T)[0] - P, x0=T_estimate(P,x), method='Newton').root

    return T, x*p.Pvap(T)/P

In [32]:
def dewT(y, P):
    T = sp.optimize.root_scalar(lambda T: dewP(y, T)[0] - P, x0=T_estimate(P,x), method='Newton').root

    return T, y*P/p.Pvap(T)


In [34]:
dewT(z, 101325)

(Array(374.69027477, dtype=float64),
 Array([0.21613614, 0.78386386], dtype=float64))

In [11]:
xs= np.linspace(0,1,21)
Ts=[]
P=101325
y1s = []
for x in np.c_[xs , 1-xs ]:
    T, y = bubbleT(x, P)
    Ts.append(T)
    y1s.append(y[0])
Ts = np.r_[Ts]
y1s=np.r_[y1s]

In [12]:
fig = make_subplots(rows=1, cols=2)
fig.add_scatter(x=xs,y=Ts, name='bubble', mode='lines', row=1,col=1)
fig.add_scatter(x=y1s,y=Ts, name='dew', mode='lines', row=1,col=1)
fig.add_scatter(x=xs,y=y1s, mode='lines', row=1,col=2, name='y1')
fig.add_scatter(x=[0,1],y=[0,1], mode='lines', row=1,col=2, name='', line_color='grey')
fig.update_layout(width=800,height=400, title=f'Benzene-Tolune VLE at P={P} Pa', xaxis_title='x, y', yaxis_title='Temperature (K)')